In [ ]:
###############################################################################
import aliases
from everest.utilities import textutils
# tuple(textutils.find_all('__getattr__', path='../everest/ptolemaic'))
# textutils.replace_all('Schematic', 'Compound', path='../everest')

In [ ]:
from everest.ptolemaic.grammar import *
import string

In [ ]:
grammar = Grammar(
    Rule('P', Clause.OneOrMore(Clause.Nonterminal('V'))),
    Rule('V', Clause.First(
        Clause.Collection(string.ascii_lowercase),
        Clause.Sequence(
            Clause.Collection('('),
            Clause.Nonterminal('P'),
            Clause.Collection(')'),
            ),
        )),
    )

In [ ]:
grammar.lower_clauses

In [ ]:
grammar.bound_clauses

In [ ]:



EMPTY = Empty()


class Container(Terminal):

    container: POS[_collabc.Container]

    def yield_clauses(self, /):
        yield self

    def match(self, parser, /):
        if parser.current_symbol in self.container:
            return 1


class Operation(Clause):

    ...


class Unary(Operation):

    clause: POS[Clause]

    def yield_clauses(self, /):
        yield self
        yield from self.clause.yield_clauses()


class OneOrMore(Unary):

    def consumptive(self, /):
        return True

    def is_seed_parent(self, clause, /):
        return clause is self.clause


class NotFollowedBy(Unary):

    def consumptive(self, /):
        return False

    def is_seed_parent(self, clause, /):
        return False


class Ennary(Operation):

    clauses: ARGS[Clause]

    @classmethod
    def _parameterise_(cls, /, *args, **kwargs):
        params = super()._parameterise_(*args, **kwargs)
        if any(cl is EMPTY for cl in params.clauses):
            raise ValueError
        params.clauses = tuple(_itertools.chain.from_iterable(
            cl.clauses if isinstance(cl, cls) else (cl,)
            for cl in params.clauses
            ))
        if len(params.clauses) < 2:
            raise ValueError
        return params

    def yield_clauses(self, /):
        yield self
        for clause in self.clauses:
            yield from clause.yield_clauses()


class Seq(Ennary):

    def consumptive(self, /):
        return True

    def is_seed_parent(self, clause, /):
        clauses = self.clauses
        if clauses[0] is clause:
            return True


class First(Ennary):

    def consumptive(self, /):
        return True

    def is_seed_parent(self, clause, /):
        return clause in self.clauses[1:]


class Rule(metaclass=_System):

    nonterminal: POS
    clause: POS[Clause]


class Grammar(metaclass=_System):

    rules: ARGS[Rule]

    def yield_clauses(self, /):
        for rule in self.rules:
            yield from rule.clause.yield_clauses()

    def sort_clauses(self, clauses, /):
        return clauses

    @prop
    def clauses(self, /):
        return tuple(self.sort_clauses(self.yield_clauses(self)))

    @prop
    def seed_parents(self, /):
        clauses = self.clauses
        seedparents = {}
        for i, clause in enumerate(clauses):
            seedparents[clause] = [
                cl for cl in clauses[:i]
                if cl.is_seed_parent(clause)
                ]
        return seedparents

    def __getitem__(self, arg: _collabc.Iterable):
        return Parser(self, arg)


MemoKey = _namedtuple('MemoKey', ('indexno', 'clauseno'))


class Match:

    __slots__ = ('memokey', 'length')

    def __init__(self, /, memokey: MemoKey, length: int):
        self.memokey, self.length = memokey, length


class MemoTable(_UserDict):

    def __getitem__(self, key: MemoKey, /):
        return super().__getitem__[tuple(key)]

    def __setitem__(self, key: MemoKey, val: int, /):
        super().__setitem__(tuple(key), val)

    def __repr__(self, /):
        return f"<{type(self).__qualname__} {super().__repr__()}>"


class Cursor:

    __slots__ = (
        '_parser', '_maxindex', '_maxclause',
        '_indexno', '_clauseno',
        )

    def __init__(self, parser, /):
        self._parser = _weakref.ref(parser)
        self._maxindex = len(self.parser.symstr)-1
        self._maxclause = len(self.parser.clauses)

    @property
    def parser(self, /):
        return self._parser()

    @property
    def indexno(self, /):
        try:
            return self._indexno
        except AttributeError:
            out = self._indexno = self._maxindex
            return out

    @indexno.setter
    def indexno(self, val, /):
        val = int(val)
        if val < 0:
            raise ValueError
        self._indexno = val
        try:
            del self.parser.current_symbol
        except AttributeError:
            pass

    @property
    def clauseno(self, /):
        try:
            return self._clauseno
        except AttributeError:
            out = self._clauseno = self._maxclause
            return out

    @clauseno.setter
    def clauseno(self, val, /):
        val = int(val)
        if val < 0:
            raise ValueError
        self._clauseno = val
        try:
            del self.parser.current_clause
        except AttributeError:
            pass

    def advance(self, /):
        if self.clauseno == 0:
            self.clauseno = self._maxclause
            self.indexno -= 1
        else:
            self.clauseno -= 1

    def __iter__(self, /):
        yield self.indexno
        yield self.clauseno

    def __repr__(self, /):
        return f"<{type(self).__qualname__} [{', '.join(map(str, self))}]>"


class Parser:

    __slots__ = (
        'grammar', 'symstr', 'memotable',
        '_clauses', '_cursor', '_terminals',
        '_current_symbol', '_current_clause',
        '__weakref__',
        )

    @property
    def clauses(self, /):
        try:
            return self._clauses
        except AttributeError:
            out = self._clauses = tuple(self.yield_clauses(self.grammar))
            return out

    def __init__(self, /, grammar: Grammar, symstr: _collabc.Iterable):
        grammar = self.grammar = grammar
        symstr = self.symstr = tuple(symstr)
        memotable = self.memotable = MemoTable()

    @property
    def clauses(self, /):
        return self.grammar.clauses

    @property
    def cursor(self, /):
        try:
            return self._cursor
        except AttributeError:
            out = self._cursor = Cursor(self)
            return out

    @property
    def terminals(self, /):
        try:
            return self._terminals
        except AttributeError:
            out = self._terminals = tuple(
                clause for clause in self.clauses
                if isinstance(clause, Terminal)
                )
            return out

    @property
    def current_symbol(self, /):
        try:
            return self._current_symbol
        except AttributeError:
            out = self._current_symbol = self.symstr[self.cursor.indexno]
            return out

    @current_symbol.deleter
    def current_symbol(self, /):
        del self._current_symbol

    @property
    def current_clause(self, /):
        try:
            return self._current_clause
        except AttributeError:
            out = self._current_clause = self.clauses[self.cursor.clauseno]
            return out

    @current_clause.deleter
    def current_clause(self, /):
        del self._current_clause

    def parse(self, /):
        memotable = self.memotable
        memotable.clear()
        for terminal in self.terminals:
            result = terminal.match(self)
            if result is None:
                continue
            print(result)
            memotable[self.cursor] = result
            break

In [ ]:
grammar = Grammar(
    Rule(
        'Foo',
        Container('abc'),
        )
    )

In [ ]:
parser = Parser(grammar, 'ccc')

In [ ]:
parser.cursor.advance()
parser.cursor

In [ ]:
parser.cursor.maxc

In [ ]:
parser.parse()

In [ ]:
parser.memotable